In [ ]:
import os

## generate trainset

In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer('../../pretrained_models/chinese-roberta-wwm-ext/vocab.txt')

In [8]:
from pypinyin import lazy_pinyin
tokens = ['今', '天', '30', '度', 'Ｉ']
lazy_pinyin(tokens)

['jin', 'tian', '30', 'du', 'Ｉ']

In [17]:
from tqdm import tqdm
from pypinyin import lazy_pinyin
# err, cor, 111, pinyin_indexes

src_path = '../../data/cn/cctc/cctc_train.tsv'
tgt_path = '../../data/cn/cctc/cctc_train.dcn.txt'

vocab = [line.strip() for line in open('../vocab/vocab.txt', 'r')]
pinyin_vocab = [line.strip() for line in open('../vocab/pinyin_vocab.txt', 'r')]

"""
英 国 卫 报 今 天 报 导 ， 为 了 避 免 引 发 英 国 国 教 派 的 不 满 ， 一 项 有 关 英 国 女 王 伊 莉 莎 白 二 世 将 在 下 新 期 访 问 梵 蒂 冈 时 ， 和 天 主 教 教 宗 若 望 保 禄 二 世 共 同 举 行 弥 撒 的 计 画 已 经 取 消 。	英 国 卫 报 今 天 报 导 ， 为 了 避 免 引 发 英 国 国 教 派 的 不 满 ， 一 项 有 关 英 国 女 王 伊 莉 莎 白 二 世 将 在 下 星 期 访 问 梵 蒂 冈 时 ， 和 天 主 教 教 宗 若 望 保 禄 二 世 共 同 举 行 弥 撒 的 计 画 已 经 取 消 。	1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1	359 107 333 10 133 318 10 60 0 333 164 14 194 358 80 359 107 107 131 231 61 21 186 0 357 341 362 103 359 107 226 332 357 167 283 7 79 292 130 368 339 344 247 82 334 81 63 92 292 0 113 318 387 131 131 396 274 332 10 178 79 292 98 322 137 345 193 275 61 127 120 357 134 257 342 0
清 华 大 学 段 海 鑫 教 授 表 示 。	清 华 大 学 段 海 新 教 授 表 示 。	1 1 1 1 1 1 1 1 1 1 1 1	254 120 56 350 72 109 344 131 293 16 292 0
"""

def B2Q(uchar):
    """单个字符 半角转全角"""
    inside_code = ord(uchar)
    if inside_code < 0x0020 or inside_code > 0x7e: # 不是半角字符就返回原来的字符
        return uchar 
    if inside_code == 0x0020: # 除了空格其他的全角半角的公式为: 半角 = 全角 - 0xfee0
        inside_code = 0x3000
    else:
        inside_code += 0xfee0
    return chr(inside_code).upper()


def pinyin_index(pinyin_str):
    try:
        res = pinyin_vocab.index(pinyin_str)
        return res
    except Exception as e:
        return 0  # [UNK]

with open(tgt_path, 'w') as f:
    for line in tqdm(open(src_path, 'r')):
        err, cor = line.strip().split('\t')
        err = ''.join([B2Q(c) for c in err])
        cor = ''.join([B2Q(c) for c in cor])
        tokens = tokenizer.tokenize(err)
        cor_tokens = tokenizer.tokenize(cor)
        assert len(tokens) == len(cor_tokens)
        len_index = ' '.join([str(1) for _ in range(len(tokens))])
        py_index = ' '.join([str(pinyin_index(_py)) for _py in lazy_pinyin(tokens)])
        res = f"{' '.join(tokens)}\t{' '.join(cor_tokens)}\t{len_index}\t{py_index}\n"
        f.write(res)

646it [00:01, 365.95it/s]


## generate testset

In [6]:
src_path = '../data/cctc/cctc_test.txt'
tgt_path = '../data/cctc/cctc.txt'


def B2Q(uchar):
    """单个字符 半角转全角"""
    inside_code = ord(uchar)
    if inside_code < 0x0020 or inside_code > 0x7e: # 不是半角字符就返回原来的字符
        return uchar 
    if inside_code == 0x0020: # 除了空格其他的全角半角的公式为: 半角 = 全角 - 0xfee0
        inside_code = 0x3000
    else:
        inside_code += 0xfee0
    return chr(inside_code).upper()


def fn(src_path, tgt_path, source='CCTC'):
    items = [line.split('\t')[0] for line in open(src_path, 'r')]
    print(len(items))
    with open(tgt_path, 'w') as f:
        for idx, item in enumerate(items):
            item = ''.join([B2Q(c) for c in item])
            f.write(f"(pid={source}-{idx})\t{item}\n")
    

# src_path = '../data/cctc/cctc_test.txt'
# tgt_path = '../data/cctc/cctc.txt'
# fn(src_path, tgt_path, 'CCTC')

src_path = '../data/findoc/findoc_collect.txt'
tgt_path = '../data/findoc/findoc.txt'
fn(src_path, tgt_path, 'FinDoc')

368
